# Xarray Dataset and DataArray operations

In [15]:
# initialization
import numpy as np
import pandas as pd
import xarray as xr

In [16]:
# Load the B-SOSE dataset
bsose = xr.open_dataset("data/bsose_monthly_velocities.nc")

## Subsetting an xarray Dataset or DataArray

The main tool to subset an xarray `Dataset` or `DataArray` is the `.isel()` and `.sel()` methods. Similar to `.iloc[]` of DataFrame, `.isel()` subset a Dataset or DataArray by indices of dimension. For example, to select the most shallow depths from the `bsose` Dataset, we may do:

In [3]:
bsose.isel(depth=0)

<xarray.Dataset> Size: 3MB
Dimensions:  (time: 12, lat: 147, lon: 135)
Coordinates:
  * time     (time) datetime64[ns] 96B 2012-01-30T20:00:00 ... 2012-12-30T12:...
  * lat      (lat) float32 588B -77.93 -77.79 -77.65 ... -31.09 -30.52 -29.94
  * lon      (lon) float32 540B 90.17 90.83 91.5 92.17 ... 178.2 178.8 179.5
    depth    float32 4B 2.1
Data variables:
    U        (time, lat, lon) float32 953kB ...
    V        (time, lat, lon) float64 2MB ...
Attributes:
    name:     B-SOSE (Southern Ocean State Estimate) model output

We can also supply a slice as arguments to `isel()` and `sel()`. However, instead of using the shorthand _start_:_stop_:_step_, we need to use an explicit `slice()` call. For example, to select the two most shallow depths from the `bsose` Dataset, we can do:

In [4]:
bsose.isel(depth=slice(0, 2))

<xarray.Dataset> Size: 6MB
Dimensions:  (time: 12, depth: 2, lat: 147, lon: 135)
Coordinates:
  * time     (time) datetime64[ns] 96B 2012-01-30T20:00:00 ... 2012-12-30T12:...
  * lat      (lat) float32 588B -77.93 -77.79 -77.65 ... -31.09 -30.52 -29.94
  * lon      (lon) float32 540B 90.17 90.83 91.5 92.17 ... 178.2 178.8 179.5
  * depth    (depth) float32 8B 2.1 26.25
Data variables:
    U        (time, depth, lat, lon) float32 2MB ...
    V        (time, depth, lat, lon) float64 4MB ...
Attributes:
    name:     B-SOSE (Southern Ocean State Estimate) model output

As you should be familiar right now, the slice is *endpoint exclusive* when you use `.isel()`

Furthermore, we can make selection on multiple dimension in a single go, e.g.:

In [5]:
bsose.isel(depth=slice(0,2), time=5)

<xarray.Dataset> Size: 477kB
Dimensions:  (depth: 2, lat: 147, lon: 135)
Coordinates:
    time     datetime64[ns] 8B 2012-06-30T23:00:00
  * lat      (lat) float32 588B -77.93 -77.79 -77.65 ... -31.09 -30.52 -29.94
  * lon      (lon) float32 540B 90.17 90.83 91.5 92.17 ... 178.2 178.8 179.5
  * depth    (depth) float32 8B 2.1 26.25
Data variables:
    U        (depth, lat, lon) float32 159kB ...
    V        (depth, lat, lon) float64 318kB ...
Attributes:
    name:     B-SOSE (Southern Ocean State Estimate) model output

While `.isel()` can be useful in limited situations, the more useful subsetting method is `.sel()`, which subset dimensions by the corresponding coordinate values, e.g.,:

In [13]:
bsose.sel(depth=2.1)

<xarray.Dataset> Size: 3MB
Dimensions:  (time: 12, lat: 147, lon: 135)
Coordinates:
  * time     (time) datetime64[ns] 96B 2012-01-30T20:00:00 ... 2012-12-30T12:...
  * lat      (lat) float32 588B -77.93 -77.79 -77.65 ... -31.09 -30.52 -29.94
  * lon      (lon) float32 540B 90.17 90.83 91.5 92.17 ... 178.2 178.8 179.5
    depth    float32 4B 2.1
Data variables:
    U        (time, lat, lon) float32 953kB ...
    V        (time, lat, lon) float64 2MB ...
Attributes:
    name:     B-SOSE (Southern Ocean State Estimate) model output

And just like `.iloc[]` for DataFrame, `.sel()` is *endpoint inclusive*

In [15]:
bsose.sel(depth=slice(2.1, 26.25))

<xarray.Dataset> Size: 6MB
Dimensions:  (time: 12, depth: 2, lat: 147, lon: 135)
Coordinates:
  * time     (time) datetime64[ns] 96B 2012-01-30T20:00:00 ... 2012-12-30T12:...
  * lat      (lat) float32 588B -77.93 -77.79 -77.65 ... -31.09 -30.52 -29.94
  * lon      (lon) float32 540B 90.17 90.83 91.5 92.17 ... 178.2 178.8 179.5
  * depth    (depth) float32 8B 2.1 26.25
Data variables:
    U        (time, depth, lat, lon) float32 2MB ...
    V        (time, depth, lat, lon) float64 4MB ...
Attributes:
    name:     B-SOSE (Southern Ocean State Estimate) model output

Quite often you may not know the exact coordinates of a dimension you want to subset. There are two mechanisms in `.sel()` that can assist you in such case. First of all, in selecting a single coordinate, you can use the `method="nearest"` argument to select the closest match, e.g.:

In [16]:
bsose.sel(depth=5, method="nearest")

<xarray.Dataset> Size: 3MB
Dimensions:  (time: 12, lat: 147, lon: 135)
Coordinates:
  * time     (time) datetime64[ns] 96B 2012-01-30T20:00:00 ... 2012-12-30T12:...
  * lat      (lat) float32 588B -77.93 -77.79 -77.65 ... -31.09 -30.52 -29.94
  * lon      (lon) float32 540B 90.17 90.83 91.5 92.17 ... 178.2 178.8 179.5
    depth    float32 4B 2.1
Data variables:
    U        (time, lat, lon) float32 953kB ...
    V        (time, lat, lon) float64 2MB ...
Attributes:
    name:     B-SOSE (Southern Ocean State Estimate) model output

Second, when you subset a Dataset or DataArray by slice, you can specify bounds are not exact coordinates, e.g.,

In [17]:
bsose.sel(depth=slice(0, 100))

<xarray.Dataset> Size: 9MB
Dimensions:  (time: 12, depth: 3, lat: 147, lon: 135)
Coordinates:
  * time     (time) datetime64[ns] 96B 2012-01-30T20:00:00 ... 2012-12-30T12:...
  * lat      (lat) float32 588B -77.93 -77.79 -77.65 ... -31.09 -30.52 -29.94
  * lon      (lon) float32 540B 90.17 90.83 91.5 92.17 ... 178.2 178.8 179.5
  * depth    (depth) float32 12B 2.1 26.25 65.0
Data variables:
    U        (time, depth, lat, lon) float32 3MB ...
    V        (time, depth, lat, lon) float64 6MB ...
Attributes:
    name:     B-SOSE (Southern Ocean State Estimate) model output

The same also works for time:

In [18]:
bsose.sel(time=slice(pd.to_datetime("2012-01-01"), pd.to_datetime("2012-03-31")))

<xarray.Dataset> Size: 5MB
Dimensions:  (time: 2, depth: 10, lat: 147, lon: 135)
Coordinates:
  * time     (time) datetime64[ns] 16B 2012-01-30T20:00:00 2012-03-01T06:00:00
  * lat      (lat) float32 588B -77.93 -77.79 -77.65 ... -31.09 -30.52 -29.94
  * lon      (lon) float32 540B 90.17 90.83 91.5 92.17 ... 178.2 178.8 179.5
  * depth    (depth) float32 40B 2.1 26.25 65.0 105.0 ... 450.0 700.0 1.1e+03
Data variables:
    U        (time, depth, lat, lon) float32 2MB ...
    V        (time, depth, lat, lon) float64 3MB ...
Attributes:
    name:     B-SOSE (Southern Ocean State Estimate) model output

Finally, just like `.loc[]` for DataFrame, you can subset in `.sel()` using logical vectors, e.g.:

In [21]:
bsose.sel(lat=(bsose.coords["lat"] < -50) & (bsose.coords["lat"] > -70))

<xarray.Dataset> Size: 12MB
Dimensions:  (time: 12, depth: 10, lat: 63, lon: 135)
Coordinates:
  * time     (time) datetime64[ns] 96B 2012-01-30T20:00:00 ... 2012-12-30T12:...
  * lat      (lat) float32 252B -69.99 -69.76 -69.53 ... -50.96 -50.53 -50.11
  * lon      (lon) float32 540B 90.17 90.83 91.5 92.17 ... 178.2 178.8 179.5
  * depth    (depth) float32 40B 2.1 26.25 65.0 105.0 ... 450.0 700.0 1.1e+03
Data variables:
    U        (time, depth, lat, lon) float32 4MB ...
    V        (time, depth, lat, lon) float64 8MB ...
Attributes:
    name:     B-SOSE (Southern Ocean State Estimate) model output

## DataArray calculations and Dataset modifications

Arithmetic operators (e.g., `+`, `-`, `*`, `/`) work as expected on xarray `DataArray`, and the result is another `DataArray`, For example, the `U` and `V` variables in `bsose` are eastward and northward ocean current velocity, respectively. Ignoring the vertical component of the ocean current, from the usual definition of speed and velocity, the ocean current speed can be obtained as:

In [3]:
speed = np.sqrt(bsose["U"]**2 + bsose["V"]**2)

In [4]:
display(speed)

<xarray.DataArray (time: 12, depth: 10, lat: 147, lon: 135)> Size: 19MB
array([[[[0.        , 0.        , 0.        , ..., 0.06010989,
          0.05197227, 0.04096739],
         [0.        , 0.        , 0.        , ..., 0.14071839,
          0.11713416, 0.09964565],
         [0.        , 0.        , 0.        , ..., 0.04750525,
          0.04322098, 0.05296802],
         ...,
         [0.05226932, 0.0536007 , 0.0527774 , ..., 0.01013198,
          0.00831877, 0.01163159],
         [0.04155295, 0.03667319, 0.03206795, ..., 0.02356772,
          0.02530149, 0.02231216],
         [0.04977216, 0.03858907, 0.026114  , ..., 0.0356411 ,
          0.02995015, 0.02251477]],

        [[0.        , 0.        , 0.        , ..., 0.05217244,
          0.04406487, 0.03266923],
         [0.        , 0.        , 0.        , ..., 0.10693451,
          0.08800546, 0.07192853],
         [0.        , 0.        , 0.        , ..., 0.05761292,
          0.05014921, 0.0393808 ],
...
         [0.01130055, 0.01364487, 0.01592921, ..., 0.00459311,
          0.00633551, 0.00740674],
         [0.01258866, 0.01473226, 0.01583225, ..., 0.00479583,
          0.00627474, 0.00715925],
         [0.01517832, 0.01638868, 0.01647329, ..., 0.00544246,
          0.00621286, 0.00667411]],

        [[0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         ...,
         [0.00782566, 0.0087817 , 0.00968277, ..., 0.00242523,
          0.00420563, 0.00568682],
         [0.00768446, 0.00835071, 0.00852652, ..., 0.00387363,
          0.00473127, 0.00590548],
         [0.00762166, 0.00788987, 0.00769931, ..., 0.00566757,
          0.00615353, 0.00707046]]]], shape=(12, 10, 147, 135))
Coordinates:
  * time     (time) datetime64[ns] 96B 2012-01-30T20:00:00 ... 2012-12-30T12:...
  * lat      (lat) float32 588B -77.93 -77.79 -77.65 ... -31.09 -30.52 -29.94
  * lon      (lon) float32 540B 90.17 90.83 91.5 92.17 ... 178.2 178.8 179.5
  * depth    (depth) float32 40B 2.1 26.25 65.0 105.0 ... 450.0 700.0 1.1e+03

Note that numpy mapping functions can be applied on xarray `DataArray` without any issues.

If we have a xarray `Dataset`, we can assign new data variables to it using the same `[]` operator. For example, with our speed calculation, we may do:

In [7]:
bsose["speed"] = np.sqrt(bsose["U"]**2 + bsose["V"]**2)

In [8]:
display(bsose)

<xarray.Dataset> Size: 48MB
Dimensions:  (time: 12, depth: 10, lat: 147, lon: 135)
Coordinates:
  * time     (time) datetime64[ns] 96B 2012-01-30T20:00:00 ... 2012-12-30T12:...
  * lat      (lat) float32 588B -77.93 -77.79 -77.65 ... -31.09 -30.52 -29.94
  * lon      (lon) float32 540B 90.17 90.83 91.5 92.17 ... 178.2 178.8 179.5
  * depth    (depth) float32 40B 2.1 26.25 65.0 105.0 ... 450.0 700.0 1.1e+03
Data variables:
    U        (time, depth, lat, lon) float32 10MB 0.0 0.0 ... -0.005353
    V        (time, depth, lat, lon) float64 19MB 0.0 0.0 ... -0.004619
    speed    (time, depth, lat, lon) float64 19MB 0.0 0.0 ... 0.006154 0.00707
Attributes:
    name:     B-SOSE (Southern Ocean State Estimate) model output

Again note that the xarray is being modified *in-place*. Moreover, if the data variable already exists it will be overwritten.

## Xarray statistics functions

In addition to calculation involving arithmetic operator and numpy mapping functions, xarray also provides a number of statistics function, which can be applied along specific dimensions. For example, suppose we want to compute yearly (month averaged) ocean current, we may do:

In [11]:
bsose_yearly = bsose.mean("time")
display(bsose_yearly)

<xarray.Dataset> Size: 4MB
Dimensions:  (depth: 10, lat: 147, lon: 135)
Coordinates:
  * lat      (lat) float32 588B -77.93 -77.79 -77.65 ... -31.09 -30.52 -29.94
  * lon      (lon) float32 540B 90.17 90.83 91.5 92.17 ... 178.2 178.8 179.5
  * depth    (depth) float32 40B 2.1 26.25 65.0 105.0 ... 450.0 700.0 1.1e+03
Data variables:
    U        (depth, lat, lon) float32 794kB 0.0 0.0 0.0 ... -0.005053 -0.00524
    V        (depth, lat, lon) float64 2MB 0.0 0.0 0.0 ... -0.0009622 -0.001767
    speed    (depth, lat, lon) float64 2MB 0.0 0.0 0.0 ... 0.006136 0.006688

Note that the same method exists for DataArray also, for example:

In [12]:
speed_yearly = speed.mean("time")
display(speed_yearly)

<xarray.DataArray (depth: 10, lat: 147, lon: 135)> Size: 2MB
array([[[0.        , 0.        , 0.        , ..., 0.05396177,
         0.04697288, 0.03941176],
        [0.        , 0.        , 0.        , ..., 0.12847457,
         0.11631608, 0.10786684],
        [0.        , 0.        , 0.        , ..., 0.10092306,
         0.08603653, 0.09591545],
        ...,
        [0.0525897 , 0.04824668, 0.04024953, ..., 0.02923318,
         0.02910917, 0.03795754],
        [0.04545898, 0.0475362 , 0.04286196, ..., 0.03544119,
         0.03534931, 0.0390486 ],
        [0.04179625, 0.04573175, 0.04619415, ..., 0.05025192,
         0.04542044, 0.04403939]],

       [[0.        , 0.        , 0.        , ..., 0.04670928,
         0.03689622, 0.02867834],
        [0.        , 0.        , 0.        , ..., 0.1105175 ,
         0.09622564, 0.08323717],
        [0.        , 0.        , 0.        , ..., 0.09972569,
         0.08335326, 0.072648  ],
...
        [0.00888233, 0.0080989 , 0.00825608, ..., 0.0045786 ,
         0.00564794, 0.00635814],
        [0.00902226, 0.00767792, 0.00720942, ..., 0.00602174,
         0.00700923, 0.00739127],
        [0.00944595, 0.00874795, 0.00834893, ..., 0.00687001,
         0.00730173, 0.00704931]],

       [[0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        ...,
        [0.00586737, 0.00581053, 0.00598294, ..., 0.00441925,
         0.00506245, 0.0060725 ],
        [0.00476857, 0.00442139, 0.00426584, ..., 0.00509038,
         0.00580745, 0.00664087],
        [0.0046224 , 0.00411261, 0.00391904, ..., 0.00555926,
         0.00613552, 0.0066876 ]]], shape=(10, 147, 135))
Coordinates:
  * lat      (lat) float32 588B -77.93 -77.79 -77.65 ... -31.09 -30.52 -29.94
  * lon      (lon) float32 540B 90.17 90.83 91.5 92.17 ... 178.2 178.8 179.5
  * depth    (depth) float32 40B 2.1 26.25 65.0 105.0 ... 450.0 700.0 1.1e+03

Other statistics functions include:
+ `.median()`: calculate the median along given dimension(s)
+ `.min()`: calculate the minimum along given dimension(s)
+ `.max()`: calculate the maximum along given dimension(s)
+ `.sum()`: calculate sum along given dimension(s)
+ `.var()`: calculate the variance along given dimension(s)
+ `.std()`: calculate standard deviation along given dimension(s)

## Remark: methods chaining

Sometimes you want to perform different action on different dimensions, e.g., you want to extract data at the shallowest depth and also average over time. Since `.sel()`, `.mean()`, etc. all return an object of the same type as the input, we can easily perform multiple actions using a coding style known as method chaining. For example, for the specific case above, we may do:

In [14]:
bsose.isel(depth=0).mean("time")

<xarray.Dataset> Size: 398kB
Dimensions:  (lat: 147, lon: 135)
Coordinates:
  * lat      (lat) float32 588B -77.93 -77.79 -77.65 ... -31.09 -30.52 -29.94
  * lon      (lon) float32 540B 90.17 90.83 91.5 92.17 ... 178.2 178.8 179.5
    depth    float32 4B 2.1
Data variables:
    U        (lat, lon) float32 79kB 0.0 0.0 0.0 0.0 ... 0.03736 0.03416 0.02836
    V        (lat, lon) float64 159kB 0.0 0.0 0.0 ... 0.02628 0.00748 0.001332
    speed    (lat, lon) float64 159kB 0.0 0.0 0.0 ... 0.05025 0.04542 0.04404